<a href="https://colab.research.google.com/github/leedhn/NewsDashboard/blob/nlp/news_clustering_classed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##DAHYUN

!git clone https://github.com/SKTBrain/KoBERT.git
%cd KoBERT
!pip install -r requirements.txt
!pip install .
%cd ..
!git clone https://github.com/BM-K/KoSentenceBERT_SKTBERT.git
%cd KoSentenceBERT_SKTBERT
!pip install -r requirements.txt

Cloning into 'KoBERT'...
remote: Enumerating objects: 155, done.
remote: Total 155 (delta 0), reused 0 (delta 0), pack-reused 155
Receiving objects: 100% (155/155), 179.81 KiB | 1.06 MiB/s, done.
Resolving deltas: 100% (82/82), done.
/content/KoBERT
     |████████████████████████████████| 55.0MB 104kB/s 
     |████████████████████████████████| 348kB 54.9MB/s 
     |████████████████████████████████| 1.1MB 48.4MB/s 
     |████████████████████████████████| 4.1MB 50.1MB/s 
     |████████████████████████████████| 1.5MB 56.8MB/s 
     |████████████████████████████████| 890kB 48.1MB/s 
     |████████████████████████████████| 2.9MB 47.2MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588527 sha256=69ca39377147d66a56c7f28a22f891edfb080628b1fba84f5b58c8f5cfc76567
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=89

In [2]:
%cd ..
# !pip install boto3
# !pip install s3fs
# !pip uninstall transformers --yes
# !pip install transformers
# !pip uninstall tokenizers --yes
# !pip install tokenizers
!pip install -r requirements.txt
%cd KoSentenceBERT_SKTBERT

/content
  Using cached https://files.pythonhosted.org/packages/0f/1c/e789a8b12e28be5bc1ce2156cf87cb522b379be9cadc7ad8091a4cc107c4/tokenizers-0.9.4-cp36-cp36m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/50/0c/7d5950fcd80b029be0a8891727ba21e0cd27692c407c51261c3c921f6da3/transformers-4.1.1-py3-none-any.whl
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 102kB 7.3MB/s 
     |████████████████████████████████| 143kB 19.0MB/s 
     |████████████████████████████████| 1.3MB 27.0MB/s 
     |████████████████████████████████| 296kB 55.4MB/s 
     |████████████████████████████████| 143kB 60.6MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3163 sha256=3280c2c7291cdf44a254c623040c4a2aa5eb459d5898c346b7cc4c897de2cf5a
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl
ERROR: requests 2.23.0 has requireme

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
##DAHYUN
#만약 SentenceTransformer경로 에러가 뜬다면...일단 import된 패키지를 지워야 하니까! 아래 주석을 돌리세요.
#del SentenceTransformer,util
#os.path.abspath(inspect.getfile(SentenceTransformer))
%ls

Clustering.py        output/           SemanticSearch.py       training_nli.py
con_training_sts.py  README.md         sentence_transformers/  training_sts.py
KorNLUDatasets/      requirements.txt  tokenizers/             transformers/


In [11]:
#%cd /KoSentenceBERT_SKTBERT/
#%ls

/content/KoSentenceBERT_SKTBERT
Clustering.py        output/           SemanticSearch.py       training_nli.py
con_training_sts.py  README.md         sentence_transformers/  training_sts.py
KorNLUDatasets/      requirements.txt  tokenizers/             transformers/


In [11]:
import boto3
import json
import pandas as pd
import s3fs
import os
import inspect
from sentence_transformers import SentenceTransformer, util
import numpy as np
from sklearn.cluster import DBSCAN
from numpy import dot
from numpy.linalg import norm
import seaborn as sns; sns.set_theme()
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import psycopg2 as pg2
from sklearn.metrics.pairwise import cosine_similarity
np.random.seed(0)

pd.set_option('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None)

eps={'오피니언':1,'세계':0.65,'사회':1.1 ,'경제':1.15 ,'생활문화':0.75 ,'IT과학':1 }
min_samples={'오피니언':3,'세계':3,'사회':5,'경제':5 ,'생활문화':3 ,'IT과학' :3 }

class Clusteringnews:
    def __init__(self):
        self.section=''
        self.df_merge= pd.DataFrame(columns=["category",
                                             "aid",
                                             "date",
                                             "title",
                                             "content",
                                             "company"])
        self.model_path = 'output/training_sts'
       

    def bring_input_data_from_s3(self,arg0,arg1): # key에 들어가는 cate와 날짜
        self.section=arg0
        s3 = boto3.client('s3',
        #key 삭제해야함 ㅠㅠ 
            aws_access_key_id='',
            aws_secret_access_key='')
        key_list=s3.list_objects_v2(Bucket='naver-news-dev')

        df_key = pd.DataFrame(columns=["name",
            "section",
            "date",
            "start_date",
            "end_date",
            "num",                               
            "key"])
        key_temp = 0
        for i in key_list['Contents']: 
            #print(i['Key'])
            try:
                name,section,date=i['Key'].split('/')
                start_date,end_date,num,trash = date.split('_')
                df_key.loc[key_temp] = [name,section,date,start_date,end_date,num,i['Key']]
                key_temp = key_temp +1
            except:
                print('error')
        df_key=df_key[(df_key['section']==arg0) &  (df_key['start_date']==arg1)]['key']
        print(df_key)
        
        for i in df_key:
            obj = s3.get_object(Bucket='naver-news-dev', Key=i)
            data = json.loads(obj['Body'].read())
            df = pd.DataFrame(data)
            df=df.transpose()
            self.df_merge=pd.concat([self.df_merge,df],axis=0)
        self.df_merge['aid'] = self.df_merge['aid'].astype(str)
        

    def bring_input_data_from_post(self,arg0): # category
        conn=pg2.connect(host="ybigta-news.cga4jkz7xhwb.ap-northeast-2.rds.amazonaws.com",user="news",password="newsdashboard!", database = 'dev',port="5432")
        cur = conn.cursor()

        query = """SELECT  case when category = 100 then '정치'
                    when category = 101 then '경제'
                    when category = 102 then '사회'
                    when category = 103 then '생활문화'
                    when category = 105 then 'IT과학'
                    when category = 110 then '오피니언' end as category
                    , id as aid, date(ni.datetime) as date, title, content, press as company
                    FROM news_info ni
                    where ni.datetime > now() - (36000 * interval '1 sec') 
                    and category = '""" + arg0 + """'
                    """
        cur.execute(query)
        rows = cur.fetchall()
        conn.commit()
        self.df_merge = pd.DataFrame(rows,columns=["category",
        "aid",
        "date",
        "title",
        "content",
        "company"])
        self.df_merge['aid'] = self.df_merge['aid'].astype(str)
        return self.df_merge


    def bring_input_data_from_mongo(arg0):
        temp_df = pd.DataFrame(arg0)
        temp_df=temp_df.transpose()
        return temp_df

    #SentenceBERT로 뉴스 제목들 임베딩해서 리턴
    def embedding(self,corpus_raw,embedder):

        
        corpus=corpus_raw[0:16]
        corpus_embeddings = embedder.encode(corpus)
        corpus=self.df_title['title'].tolist()[16:32]

        corpus_raw_16=len(corpus_raw)//16
        for i in range(corpus_raw_16-1):
            corpus=corpus_raw[16*(i+1):16*(i+2)]
            corpus_embeddings=np.concatenate((corpus_embeddings,embedder.encode(corpus)),axis=0)
        if (len(corpus_raw)>corpus_raw_16*16):
            corpus=corpus_raw[corpus_raw_16*16:] 
        corpus_embeddings=np.concatenate((corpus_embeddings,embedder.encode(corpus)),axis=0)
        
        return corpus_embeddings

    #클러스터 내 코사인유사도 구해서 리스트로 반환
    def cs_score(self,gc,gc_id,gc_raw):
        cs_score=0
        self.cs_scores=[0 for i in range(len(gc))]

        for i in range(len(gc)):
            cs_score=0
            cs_score=sum(sum(cosine_similarity(gc[i],gc[i])))-len(gc[i])
            if len(gc[i])==1:
                self.cs_scores[i]=cs_score
            else:
                self.cs_scores[i] =cs_score/((len(gc[i])*(len(gc[i])-1)))#/2)
        return self.cs_scores

    #클러스터-리스트 리턴
    #프린트문 나중에 지울 것.
    def cluster_list(self,gc_id,gc_raw, cs_scores):

        final_cluster_raw=[]
        final_cluster_news_id=[]
        final_cluster_cs_score=[]

        for i in range(len(gc_id)):
            if cs_scores[i]<0.95:
                continue

            final_cluster_news_id.append(gc_id[i]) #최종으로 살아남은 클러스터 별 뉴스 id
            final_cluster_raw.append(gc_raw[i]) #최종으로 살아남은 클러스터별 뉴스 제목
            final_cluster_cs_score.append(cs_scores[i]) #최종으로 살아남은 클러스터 별 코사인 유사도
        
        return final_cluster_raw,final_cluster_news_id,final_cluster_cs_score    
    
    #DAHYUN
    #클러스터 정렬 후 딕셔너리로 리턴.
    def make_cluster_dic(self,final_cluster_raw,final_cluster_news_id,final_cluster_cs_score):
        print('here')
        cluster_dic_raw={}
        cluster_dic={}
        for i in range(len(final_cluster_raw)):
            cluster_dic_raw[i]={}
            cluster_dic_raw[i]['id']=final_cluster_news_id[i]
            cluster_dic_raw[i]['cs_score']=final_cluster_cs_score[i]
            cluster_dic_raw[i]['size']=len(final_cluster_news_id[i])    
        #클러스터 딕셔너리를 뉴스기사 개수 순으로 정렬해요
        cluster_dic_raw=sorted(cluster_dic_raw.items(),key=(lambda x: x[1]['size']),reverse=True)
        
        for i in range(len(final_cluster_news_id)):
            cluster_dic[i]={}
            cluster_dic[i]['id']=cluster_dic_raw[i][1]['id']
            cluster_dic[i]['cs_score']=cluster_dic_raw[i][1]['cs_score']
            cluster_dic[i]['size']=cluster_dic_raw[i][1]['size']
        print(len(final_cluster_news_id))
        return cluster_dic

    #실제로 클러스터링 하는 메인 함수
    def clustering_function(self):
        
        #SentenceTransformer 모델 정의 = embedder 
        embedder = SentenceTransformer(self.model_path)

        #df_title 정의
        df_t=self.df_merge.copy()
        self.df_title=df_t[['category','aid','date','title']]

        #corpus_raw: 뉴스 제목들 리스트
        corpus_raw=self.df_title['title'].tolist()

        #뉴스 제목들 embedder 사용해서 벡터표현으로 바꿈.
        print('embedding start')
        corpus_embeddings=self.embedding(corpus_raw, embedder)
        print('embedding finish')
        #PCA
        pca2 = PCA(n_components = 2)                #for plt
        X2D = pca2.fit_transform(corpus_embeddings) #for plt
        pca3 = PCA(n_components = 10)
        X3D = pca3.fit_transform(corpus_embeddings)

        #표준화(평균=0, 분산=1)
        scale = StandardScaler()                    #for plt
        scale.fit(X2D)                              #for plt
        scaled_X = scale.transform(X2D)             #for plt
        scaled_x = scaled_X[:,0]                    #for plt
        scaled_y = scaled_X[:,1]                    #for plt

        #DBSCAN
        #parameters-----------------------------------------------
        eps={'오피니언':1,'세계':0.65,'사회':1.1 ,'경제':1.15 ,'생활문화':0.75 ,'IT과학':1 }
        min_samples={'오피니언':3,'세계':5,'사회':5,'경제':5 ,'생활문화':3 ,'IT과학' :3 }

        #print('dbscan start')
        dbscan = DBSCAN(eps=eps[self.section], min_samples=min_samples[self.section]) #사회.

        cluster = dbscan.fit_predict(X3D)
        #print('dbscan finish')

        #클러스터링 된 거 plt로 점 찍어서 보여줌.-----------------
        #plt.scatter(x=scaled_x,y=scaled_y,c=cluster)
        
        #print(max(cluster)) #클러스터 개수
        #print(len(cluster)) #should be matched with news number

        #클러스터링 하는 데에 사용----------------------------
        group_cluster=[[] for i in range(len(cluster))] 
        group_raw=[[]for i in range(len(cluster))] 
        group_news_id=[[]for i in range(len(cluster))]
        #-----------------------------------------------

        gc=[] #벡터표현 들어감 (클러스터링 된 리스트)
        gc_raw=[] #뉴스 제목 한글로 들어감 (클러스터링 된 리스트)
        gc_id=[] #뉴스 아이디 (클러스터링 된 리스트)

        np_ids=self.df_title['aid'].tolist()
        #-----------------------------------------------
        for i in range(len(corpus_raw)):
            group_cluster[cluster[i]].append(X3D[i])
            group_raw[cluster[i]].append(corpus_raw[i])
            group_news_id[cluster[i]].append(np_ids[i])

        num=0 #클러스터링 된 뉴스 개수들. 
        for i in range(len(cluster)):
            if i==len(cluster)-1 or group_cluster[i]==[]:
                continue
            gc.append(group_cluster[i])
            gc_raw.append(group_raw[i])
            gc_id.append(group_news_id[i])
            num+=len(group_raw[i])

        #DBSCAN으로 클러스터링
        #각 클러스터별로 코사인유사도 구해서 0.95이상만 살림
        #클러스터별 코사인 유사도는 cs_scores리스트에 저장됨. 클러스터 인덱스와 같은 인덱스를 사용한다.
        #클러스터별로 묶인 벡터 리스트는 gc
        #클러스터 별로 묶인 벡터의 원본 스트링(기사제목)은 gc_raw에 저장되어있어요.
        #print('cs score calculation start')
        cs_scores=self.cs_score(gc,gc_id,gc_raw)
        #print('cs score calculation finish')
        
        final_cluster_raw,final_cluster_news_id,final_cluster_cs_score=self.cluster_list(gc_id,gc_raw,cs_scores)

        self.cluster_dic=self.make_cluster_dic(final_cluster_raw,
                                          final_cluster_news_id,
                                          final_cluster_cs_score)
        #print('clustering finish')
        return (self.cluster_dic)

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [13]:
def get_section_dic(sections,date):
    cn=Clusteringnews()
    section_dic={}

    for section in sections:
        cluster_dic={}
        cn.bring_input_data_from_s3(section,date)
        cluster_dic=cn.clustering_function()

        section_dic[section]=cluster_dic
    return section_dic

In [ ]:
sections=['IT과학','사회','세계','오피니언','경제','생활문화']
section_dic=get_section_dic(sections,'20200922')

error
error
error
10    news_content/IT과학/20200922_20200923_0_9361.json
11    news_content/IT과학/20200922_20200923_1_9361.json
12    news_content/IT과학/20200922_20200923_2_9361.json
Name: key, dtype: object
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
